In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def preprocess(row):
    row = re.sub('ㅁ', "", row)
    row = re.sub(r"\n", ",", row)
    
    # 용어
    row = re.sub("YoY|yoy|YOY", "전년 대비 증감율", row)
    row = re.sub("QoQ|qoq|QOQ", "전분기 대비 증감율", row)
    row = re.sub("MoM|mom|MOM", "전월 대비 증감율", row)
    row = re.sub("YTD", "연초 누계", row)
    row = re.sub("MTD", "월초 누계", row)
    
    # 분기 표현 1Q21->21년도 1분기
    quarter_rule = re.compile(r'([1-4]Q[\d]{2})')
    quarter_text = quarter_rule.finditer(row)
    startIdxList = []
    for text in quarter_text :
        startIdxList.append(text.start())
    for i in range(len(startIdxList)-1, -1, -1) :
        idx = startIdxList[i]
        quar = row[idx:idx+4]
        new_quar_text = row[idx+2:idx+4]+"년도 "+row[idx]+"분기 "
        row = row.replace(quar, new_quar_text)
        
    # [내용] 제거
    row = re.sub(r'\[[^)]*\]', '', row)
        
    # 다중 띄어쓰기
    row = re.sub(' +', ' ', row)
    
    # 앞뒤 공백
    row = re.sub(r"^\s+", "", row, flags=re.UNICODE)
    row = re.sub(r"\s+$", "", row, flags=re.UNICODE)
    
    return row

In [3]:
filename = "./datasets/combined_dataset.xlsx"
# filename = "./report.xlsx"
df = pd.read_excel(filename)
df.head()

,Unnamed: 0,date,title,press,link
0,0,20210601,"삼성전자-반도체가 핵심, 투자의견 Buy, 목표주가 95,000원 제시, 3Q21까...",NaN,NaN
1,1,20210601,"삼성전자-PC?모바일 to 서버로의 수요 성장의 축 이동, 2Q21 영업이익 10....",NaN,NaN
2,2,20210601,"삼성전자-DRAM 공급 증가, DRAM 공급 증가로 2022년 수급 부담 예상, 2...",NaN,NaN
3,3,20210601,"삼성전자-수익성 위주의 신중한 공급증가, 공급 경쟁 재점화를 차단하는 선두업체의 보...",NaN,NaN
4,4,20210603,"결국 ‘4번 타자’는 삼성전자?… 삼전 급반등, 코스피 다시 3250선 눈앞",NaN,NaN


In [4]:
# print(df.isnull().sum())
# print(df)

logCnt = 1000
plusCnt = 1000
for i in range(len(df)) :
    if(i == logCnt) :
        print(i)
        logCnt += plusCnt
#     print(i, df['title'][i])
    
    # 영어로만 구성된 row는 drop
    if(len(re.findall(u'[가-힣]+', df['title'][i]))>0) :
        df['title'][i] = preprocess(df['title'][i])
    else :
        df = df.drop(i)

df['title'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['title'])
df = df.reset_index(drop=True)
print("Done!")
# print(df[250:260])

<ipython-input-4-7d6fdca842c1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = preprocess(df['title'][i])


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
Done!


In [5]:
df.to_excel("./datasets/combined_dataset_cleaned.xlsx", index = False)